# Vibrations in a 1D solid

This notebook is to accompany the _Vibrations_ content.

Version 1.0, updated 26/08/2021 by AJM

## Import packages

To streamline operations in Python, packages can be imported to perform a host of various tasks. To make this process as simple as possible, all the required packages are included in the file _[SSP.py](https://github.com/Andy-UTAS/Solid-state/blob/master/SSP.py)_ and thus we can import all of the content: 

In [ ]:
from SSP import *

## Inter-particle potential

Let's plot out the potential, although we need to include a realistic potential, so we are going to use the [Lennard-Jones potential](https://en.wikipedia.org/wiki/Lennard-Jones_potential), but the model is not super critical given we are going to approximate it!

In [ ]:
# Lennard-Jones potential
def potential(x):
    return 4 * (x ** -12 - x ** -6)

# Set min and max x parameters
x_min, x_max = [.75, 2]

# Create equally spaced points between x_min and x_max
deltax = np.linspace(x_min, x_max, 1000)

# Make the plot
fig, ax = plt.subplots()
ax.hlines(0, 0, 3, label = '$E = 0$', linestyle = '--', color = 'C0') # E = 0
ax.plot(deltax, potential(deltax), label='Potential', color = 'C1') # Potential
ax.set_xlabel('$\Delta x$')
ax.set_ylabel('Energy')
ax.set_xlim(x_min, x_max)
ax.set_ylim(-1.5,1.5)
ax.legend()

ax.set_title('Inter-particle potential');

plt.savefig('3-1-potential.svg', facecolor='white', transparent=False)

plt.show()

Now let's see how well we can approximate the low-energy well with a parabola

In [ ]:
energy = 0.25 # Offset from the bottom of the well
height = (-1+energy) # Well height

# Select the appropriate values to fit
deep = potential(deltax) < height # Select values of the potential less than the height, returns boolean array
well_x = deltax[deep] # map the x values using the boolean array
well_V = potential(deltax)[deep]  # map the potential values using the boolean array

# Define the function to fit (a cubic)
def parabola(x, a, b, c):
    return a * x**2 + b * x + c

fit = curve_fit(parabola, well_x, well_V) # perform the fit
params = fit[0] # extract the fit parameter. Note, if you haven't seen multiparameter fitting, observe that params is an array, not a value

# Make the plot
fig, ax = plt.subplots()
ax.hlines(height,0,3, label = '$E = E_{\mathrm{min}} + k_{\mathrm{B}}T$', linestyle = '--', color = 'C0')
ax.hlines(-1 ,0,3, label = '$E = E_{\mathrm{min}}$', color = 'C0')
ax.plot(deltax, potential(deltax), label='Potential', color = 'C1')
ax.plot(well_x, well_V, label='Lowest energy well', color = 'C2')

# NOTE: to get the list of fitting parameters into the model, we need to tell python to interpret the array as the list of function inputs, hence the *
ax.plot(deltax, parabola(deltax, *params), label=f'Fit', color = 'C3', linestyle = '--') 

ax.set_xlabel('$\Delta x$')
ax.set_ylabel('Energy')
ax.set_xlim(.75,2)
ax.set_ylim(-1.1,.1)
ax.legend(loc = 7)

# Put in the k_B T
ax.text(.85, 1.025 * (height - 1)/2 , r'$k_{\mathrm{B}}T$',
         horizontalalignment='center', fontsize=16)

ax.annotate("", xy=(.8, -1),
            xytext=(.8, height),
            arrowprops=dict(arrowstyle="<->"))

plt.savefig('3-1-potentialwithapprox.svg', facecolor='white', transparent=False)

plt.show()

## Dispersion relation

In [ ]:
kappa = 1
m = 1
a = 1

def dispersion(k):
    return 2 * np.sqrt(kappa/m) * abs(np.sin(k * a / 2))

scale = 3

brillouin = np.linspace(-np.pi/a, np.pi/a, 500)
ks = brillouin * scale

fig, ax = plt.subplots()
ax.plot(ks, dispersion(ks), label='Dispersion', linestyle = '--', color = 'C0')
ax.plot(brillouin, dispersion(brillouin), label='Brillouin Zone', color = 'C0')

ax.set_xlabel('$k$')
ax.set_ylabel('$\omega$')
ax.set_xlim(min(ks),max(ks))
ax.set_title('Dispersion relation');
ax.set_aspect(3) # if adjusting the aspect ratio, make sure to use bbox_inches='tight' when saving!

plt.savefig('3-1-dispersion.svg', facecolor='white', transparent=False, bbox_inches='tight')

plt.show()

Plot only the Brillouin zone

In [ ]:
fig, ax = plt.subplots()
ax.plot(brillouin, dispersion(brillouin), label='Brillouin Zone', color = 'C0')

xvals = [-np.pi/a, np.pi/a]
for v in xvals:
    ax.axvline(x=v, linestyle = '--')
    
    offset = 0.75
    if v < 0:
        sign = '-'
    elif v > 0:
        sign = '+'
        offset = -offset
        
    # Label the Brillouin range
    ax.text(v + offset, .1 , f'$k ={sign}\pi/a$',
             horizontalalignment='center', fontsize=16)

ax.set_xlabel('$k$')
ax.set_ylabel('$\omega$')

ax.hlines(2*np.sqrt(kappa/m), *xvals, label = '$E = E_{\mathrm{min}}$', color = 'C0', linestyle = '--')
# Put in the k_B T
ax.text(1*offset, 1.8*np.sqrt(kappa/m) , f'$\omega =2'+r'\sqrt{\frac{\pi}{a}}$',
         horizontalalignment='center', fontsize=16)

ax.set_aspect(2) # if adjusting the aspect ratio, make sure to use bbox_inches='tight' when saving!

draw_classic_axes(ax)
plt.savefig('3-1-brillouin.svg', facecolor='white', transparent=False, bbox_inches='tight')

plt.show()

## Aliasing 

In [ ]:
x = np.linspace(-.2, 5*a, 500)
x_max = x.max()
fig, ax = plt.subplots()

k = np.pi/ (a-a/8.624)

ax.plot(x, np.cos(k*x), label=f'$k=k_0$')
ax.plot(x, np.cos((k+2*np.pi/a)*x), label=f'$k=k_0 + 2\pi/a$')

sites = np.linspace(0,x_max,6)
ax.scatter(sites, np.cos(k*sites), c='k', s=250, zorder=5)

ax.set_xlabel('$x$')
ax.set_ylabel('$\delta x_n$')
ax.set_xlim((-.1, x_max))
ax.set_ylim((-1.3, 1.3))
ax.legend(loc='lower right')
draw_classic_axes(ax)
ax.annotate(text='', xy=(0, -1.1), xytext=(a, -1.1),
            arrowprops=dict(arrowstyle='<->', shrinkA=0, shrinkB=0))
ax.text(.5, -1.25, '$a$', ha='center');

plt.savefig('3-1-aliasing.svg', facecolor='white', transparent=False)

plt.show()